In [2]:
%matplotlib widget
from helper import *

In [3]:
# function to compute the quintic polynomial used to blend between two translational position profiles
def quinticPolynomial(pos0, pos1, vel0, vel1, acc0, acc1, t, T):
    # Calculate coefficients of the quintic polynomial
    h = pos1 - pos0

    p0 = pos0
    p1 = vel0
    p2 = 0.5*acc0
    p3 = 1/(2*T**3)*(20*h - (8*vel1 + 12*vel0)*T - (3*acc0 - acc1)*T**2)
    p4 = 1/(2*T**4)*(-30*h + (14*vel1 + 16*vel0)*T + (3*acc0 - 2*acc1)*T**2)
    p5 = 1/(2*T**5)*(12*h - 6*(vel1 + vel0)*T + (acc1 - acc0)*T**2)

    # Calculate position using the quintic polynomial
    pos = p0 + p1*t + p2*t**2 + p3*t**3 +p4*t**4 + p5*t**5
    return pos

# Define function to get an example position command with uncontinous velocity and acceleration
def get_pos(v0, a0, v1, a1, time):
    y = []
    for t in time:
        if t <= 2:
            y.append(v0*(t-2) + 0.5*a0*(t-2)**2)
        else:
            y.append(v1*(t-2.0) + 0.5*a1*(t-2.0)**2)
    return y

# Define function to calculate initial and final positions and velocities for blending
def get_p0_p1_v0_v1_blend(v0, a0, v1, a1, h):
    
    p0_blend = v0*(-h*0.5) + 0.5*a0 * (-h*0.5)**2
    p1_blend = v1*(h*0.5) + 0.5*a1*(h*0.5)**2
    
    v0_blend = v0 + a0 * (-0.5*h)
    v1_blend = v1 + a1 * (0.5*h)
  
    return p0_blend, p1_blend, v0_blend, v1_blend

# Define function to calculate blended portion of the position profile
def get_blend(v0, a0, v1, a1, time, h):
    blend_start = 2 - 0.5*h
    blend_end = 2 + 0.5*h
    
    pos0, pos1, vel0, vel1 = get_p0_p1_v0_v1_blend(v0, a0, v1, a1, h)
    y = []
    x = []
    for t in time:
        if t >= blend_start and t <= blend_end:
            x.append(t)
            y.append(quinticPolynomial(pos0, pos1, vel0, vel1, a0, a1, t - blend_start, h))

    return x, y

# Setup the figure and axis for the plot
plt.ioff()
fig = plt.figure(figsize=(8, 4))
ax = fig.add_subplot()
ax.set_ylim([-2, 2])
ax.set_title("Example of position blending")

# Initialize parameters for the sliders
h_blend = 1
v0 = 1
a0 = 0
v1 = 2
a1 = -1

# Create time points and generate initial position profiles
time = np.linspace(0, 4, 500)
pos = get_pos(v0, a0, v1, a1, time)
time_blend, pos_blend = get_blend(v0, a0, v1, a1, time, h_blend)

# Plot the initial position profile and the blended profile
line, = ax.plot(time, pos, '--', label = "unblended position profile")
line_blend, = ax.plot(time_blend, pos_blend, '-', label = "blending section")
ax.legend()
# Add vertical lines to indicate the blending region
vline1 = ax.axvline(x=2-h_blend*0.5, color='grey', linestyle='--')
vline2 = ax.axvline(x=2+h_blend*0.5, color='grey', linestyle='--')

# Create sliders for interactive control
slider_h_blend = create_slider("h_blend", h_blend, 0.2, 2)
slider_v0 = create_slider("v0", v0, -2, 2)
slider_a0 = create_slider("a0", a0, -5, 5)
slider_v1 = create_slider("v1", v1, -2, 2)
slider_a1 = create_slider("a1", a1, -5, 5)

# Define function to update the plot based on slider input
def update_plot(change):
    # update values based on slider input
    h_blend = slider_h_blend.value
    pos = get_pos(slider_v0.value, slider_a0.value, slider_v1.value, slider_a1.value, time)
    time_blend, pos_blend = get_blend(slider_v0.value, slider_a0.value, slider_v1.value, slider_a1.value, time, h_blend)
    
    # update plot data
    vline1.set_xdata([2-h_blend*0.5])
    vline2.set_xdata([2+h_blend*0.5])
    line.set_data(time, pos)
    line_blend.set_data(time_blend, pos_blend)
    
    #redraw canvas
    fig.canvas.draw()
    fig.canvas.flush_events()

# Set up observers for the sliders
slider_h_blend.observe(update_plot, names='value')
slider_v0.observe(update_plot, names='value')
slider_a0.observe(update_plot, names='value')
slider_v1.observe(update_plot, names='value')
slider_a1.observe(update_plot, names='value')

# Arrange the plot and sliders in the Jupyter notebook layout
widgets.AppLayout(
    center=fig.canvas,
    footer=widgets.VBox([slider_h_blend, slider_v0, slider_a0, slider_v1, slider_a1]),
    pane_heights=[0, 2, 1]
)

AppLayout(children=(VBox(children=(FloatSlider(value=1.0, description='h_blend', layout=Layout(width='60%'), m…